In [1]:
from dolfin import *
import numpy as np

import cell_computation as com
import cell_prop as ce
import cell_material as ma

In [2]:
import logging
logging.getLogger('FFC').setLevel(logging.WARNING)

## Setting

In [3]:
mesh = Mesh(r"m.xml")
# mesh = Mesh(r"m_fine.xml")

cell = ce.unit_cell(mesh)

inc = ce.Inclusion_Circle()
inc = [inc]
cell.inclusion(inc)

In [4]:
E_m,nu_m,E_i,nu_i = 10.0, 0.3, 1000.0, 0.3

mat_m = ma.st_venant_kirchhoff(E_m, nu_m)
mat_i = ma.st_venant_kirchhoff(E_i, nu_i)
mat_li = [mat_m, mat_i]

In [5]:
VFS = VectorFunctionSpace(cell.mesh, "CG", 1, constrained_domain=ce.PeriodicBoundary_no_corner())
strain_space = TensorFunctionSpace(cell.mesh, 'DG', 0)

In [6]:
w = Function(VFS)

In [7]:
def deform_grad_with_macro(F_bar, w_component):
    return F_bar + grad(w_component)

## Computation Template

In [8]:
def FD_C_eff(F_bar):
    F_bar = [F_bar[0,0], F_bar[0,1], F_bar[1,0], F_bar[1,1]]
    
    
    comp = com.MicroComputation(cell, mat_li, 
                                [deform_grad_with_macro],
                                [strain_space], 0)
    comp.input([F_bar], [w])
    comp.comp_fluctuation()

    return comp.avg_merge_stress()

## Computation with FD

$\mathbb{C}_{1111} \qquad  \mathbb{C}_{1211} \qquad \mathbb{C}_{2111} \qquad  \mathbb{C}_{2211}$

In [9]:
F_bar = [0.9, 0., 0., 1.]

In [11]:
delta = [0.01, 0.01/2, 0.01/4, 0.01/8]

In [12]:
# sample_num = 8
# delta = np.logspace(-2,-4,num=sample_num)

In [13]:
F_bar = np.array(F_bar).reshape(2,2)

In [15]:
for d in delta:
    F_minus = F_bar.copy()
    F_minus[0,0] = 0.9 - d/2
    F_plus = F_bar.copy()
    F_plus[0,0] = 0.9 + d/2
    
    P_minus = FD_C_eff(F_minus)
    P_plus  = FD_C_eff(F_plus)
    
    print (P_plus - P_minus)/d
    print '\n'

fluctuation computation finished
strain computation finished
average merge stress computation finished
fluctuation computation finished
strain computation finished
average merge stress computation finished
[  1.06007667e+01   3.06884647e-04   3.73899641e-04   5.96468494e+00]


fluctuation computation finished
strain computation finished
average merge stress computation finished
fluctuation computation finished
strain computation finished
average merge stress computation finished
[  1.06003904e+01   3.06940046e-04   3.73948623e-04   5.96473132e+00]


fluctuation computation finished
strain computation finished
average merge stress computation finished
fluctuation computation finished
strain computation finished
average merge stress computation finished
[  1.06002963e+01   3.06953907e-04   3.73960880e-04   5.96474292e+00]


fluctuation computation finished
strain computation finished
average merge stress computation finished
fluctuation computation finished
strain computation finished
av

## Homogenization Method Result

In [18]:
F_bar = [0.9, 0., 0., 1.]
comp = com.MicroComputation(cell, mat_li, [deform_grad_with_macro], [strain_space], 0)

comp.input([F_bar], [w])
comp.comp_fluctuation()
print comp.effective_moduli_2()

fluctuation computation finished
strain computation finished
average merge moduli computation finished
[[  1.06002649e+01   3.06957941e-04   3.73964877e-04   5.96474679e+00]
 [  3.06957941e-04   3.09779890e+00   4.16921064e+00  -1.39407196e-04]
 [  3.73964877e-04   4.16921064e+00   3.02758901e+00  -1.84507395e-04]
 [  5.96474679e+00  -1.39407196e-04  -1.84507395e-04   1.76183590e+01]]
